In [ ]:
from openpyxl import Workbook
from openpyxl.chart import ScatterChart, Series, Reference
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.drawing.line import LineProperties
from openpyxl.drawing.colors import ColorChoice

def export_freight_chart_excel(df, site, commodity, output_file):
    df = df[[
        'quantity_sqyd',
        'vendor_estimated_cost',
        'actual_freight_cost',
        'ftl_cost',
        'ltl_only_cost',
        'optimal_cost'
    ]].copy()

    wb = Workbook()
    ws = wb.active
    ws.title = "Freight Chart"

    for row in dataframe_to_rows(df, index=False, header=True):
        ws.append(row)

    chart = ScatterChart()
    chart.title = f"Freight Cost Overlay - {site} / {commodity}"
    chart.x_axis.title = "Quantity (SQYD)"
    chart.y_axis.title = "Freight Cost ($)"
    chart.legend.position = "t"
    chart.width = 20
    chart.height = 12

    max_row = ws.max_row
    x_values = Reference(ws, min_col=1, min_row=2, max_row=max_row)

    def add_series(title, y_col, marker=None, color=None):
        y_values = Reference(ws, min_col=y_col, min_row=2, max_row=max_row)
        series = Series(y_values, x_values, title=title)
        line = LineProperties()
        if color:
            line.solidFill = ColorChoice(prstClr=color)
        series.graphicalProperties.line = line
        if marker:
            series.marker.symbol = marker
            series.graphicalProperties.line.noFill = True
        chart.series.append(series)

    add_series("Vendor Cost Curve", 2, color="black")
    add_series("Actual Invoice Freight", 3, marker="circle", color="blue")
    add_series("FTL Cost", 4, color="gray")
    add_series("LTL Only Cost", 5, marker="diamond", color="navy")
    add_series("Optimal Cost", 6, marker="square", color="lightBlue")

    ws.add_chart(chart, "H2")
    wb.save(output_file)
    print(f"Chart saved to: {output_file}")